## Содержание
```
1 Прямой проход и обратное распространение
2 Разбиение набора данных
3 Нормализация данных
4 Валидация и потеря
5 Схождение
6 Контрольные точки и ранняя остановка
7 Гиперпараметры
8 Инвариантность
9 Сырые наборы данных
10 Сохранение/Восстановление модели
```

## 1 Прямой проход и обратное распространение

#### Прямой проход

<img src="images/forward_pass.png" alt="forward" height=60% width=60%>

```
    Данные подаются в нейронную сеть пакетами (batch-ами). Пакет может состоять из одного или нескольких выбранных наугад примеров из тренировочных данных.
    Тренировочные данные подаются в модель несколько раз. Всякий раз, когда мы подаем все тренировочные данные целиком, это
называется эпохой. Каждая эпоха представляет собой отличающуюся случайную перестановку примеров в пакетах – то есть не существует двух эпох с одинаковым порядком следования примеров.

```

#### Историческая справка
```
    В  ранних нейронных сетях, таких как персептрон и пр., исследователи экспериментировали со способами обновления весов, чтобы получать правильный ответ.
    Когда они работали всего с несколькими нейронами и простыми задачами, первым подходом было случайное обновление весов. Кто бы мог подумать, но случайная догадка работала. Однако, такой подход не масштабировался на большое количество нейронов (скажем, тысячи) и реально существующие приложения; на то, чтобы сделать правильную случайную догадку, могли уйти тысячи лет
    Следующим логическим шагом было попытаться сделать случайное значение пропорциональным удаленности от правильного значения. Другими словами, чем больше ошибка (удаленнее предсказание), тем больше диапазон случайных значений; и чем ближе, тем меньше диапазон. Это сокращает время поиска (весов нейронной сети) до сотен лет.
    Но такой подход не работал с многослойными нейронными сетями. Обнаружилось, что при наличии нескольких слоев эта методика приводит
к тому, что "левая рука" – один слой – отменяет работу "правой руки" – другого слоя.
```

#### Обратное распространение

<img src="images/back_prop.png" alt="backpropagation" height=60% width=60%>

```
    После того как каждый пакет тренировочных данных пропущен через модель в прямом направлении и вычислена потеря, потеря распространяется через модель в обратном направлении. Мы идем слой за слоем, обновляя параметры модели (веса и  смещения), начиная с  верхнего слоя (выхода) и  продвигаясь к  нижнему слою (входу).
    Общий метод обновления параметров основан на градиентном спуске. Оптимизатор – это имплементация градиентного спуска, задача которого состоит в обновлении параметров для минимизации потери (максимального приближения к  правильному ответу) в последующих пакетах.
```

## 2 Разбиение набора данных

#### Тренировочный и тестовый наборы

<img src="images/train_test.png" alt="train_data" height=60% width=60%>

```
     По умолчанию будем рассматривать наборы данных, которые достаточно велики и разнообразны, чтобы представлять выборочное распределение (моделируемую популяцию), и очищены (не зашумлены).
```

```
    Имея в своем распоряжении набор данных, следующим шагом будет его разбиение на примеры, которые будут использоваться для тренировки, и те, которые будут использоваться для тестирования (т.н. отложенные). Мы тренируем модель той порцией набора данных, которые являются тренировочными. Если допустить, что тренировочные данные являются хорошим выборочным распределением, то точность на тренировочных данных должна отражать точность, получаемую при развертывании в реально существующей среде (на примерах, не встречавшихся моделью во время тренировки). Но как проверить истинность этого утверждения до того, как модель будет развернута? Отсюда и вытекает предназначение тестовых данных. Мы откладываем часть данных, с помощью которой будем тестировать модель, после того как она будет обучена, чтобы убедиться, что мы получили или не получили нужную точность.
    Сколько данных следует откладывать на тренировку и тестирование? Исторически сложилось эмпирическое соотношение 80/20: 80% для тренировки и 20% для тестирования. Но пропорция 90/10 тоже хорошо работает.
```

#### Пример

```
    Для классифицирования изображений есть несколько хорошо известных, подготовленных наборов данных, таких как MNIST, CIFAR-10/100*, SVHN, Flowers, Cats vs. Dogs.
    Рассмотрим один из них

* аббр. от англ. Canadian Institute for Advanced Research – Канадский институт перспективных исследований

```

```python
import torch
import torchvision

# MNIST

# Downloading the MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root="./MNIST/train", train=True,
    transform=torchvision.transforms.ToTensor(),
    download=False)

test_dataset = torchvision.datasets.MNIST(
    root="./MNIST/test", train=False,
    transform=torchvision.transforms.ToTensor(),
    download=False)
```

```
train_dataset - 60000 imgs
test_dataset  - 10000 imgs
```

<img src="images/MNIST_data.png" alt="mnist_dataset" height=60% width=60%>

``` python
def encode_label(j):
    # 5 -> [[0], [0], [0], [0], [0], [1], [0], [0], [0], [0]]
    e = np.zeros((10,1))
    e[j] = 1.0
    return e

def reshape_data(data):
    features = [np.reshape(x[0][0].numpy(), (784,1)) for x in data]
    #print('features\n', len(features[0]))
    labels = [encode_label(y[1]) for y in data]
    #print('labels\n', len(labels[0]))
    return zip(features, labels)
```

## 3 Нормализация данных

#### Нормализация данных

```
    Каждое изображение датасета MNIST является 28×28-матрицей целочисленных значений от 0 до 255. Если посмотреть на параметры
натренированной модели (веса и смещения), то они представляют очень малые числа, обычно от –1 до 1. Проблема здесь в том, что
большие входные значения (вплоть до 255), будут порождать крупные числа по мере их умножения по всем слоям. Поэтому сети потребуется больше времени на то, чтобы усвоить свои оптимальные значения – если она вообще их усвоит.
```

Поэтому:
```python
import numpy

maxi = np.max(x_train) # 255.0
x_train = (x_train / maxi).astype(np.float32)
```

#### Стандартизация

```python
mean = np.mean(x_train)
std = np.std(x_train)
x_train = ((x_train – mean) / std).astype(np.float32)

# например x_train = ((x_train / 127.5) – 1).astype(np.float32)

```

## 4 Валидация и "потеря"

#### Валидация

<img src="images/valid.png" alt="validation" height=60% width=60%>

```
    Допустим, тренировка модели занимает несколько часов. Что делать, если мы не хотим ждать до конца тренировки чтобы узнать точность модели? Для этого мы выделяем небольшую часть тренировочных данных, которые называем валидационными данными.
    Модель не тренируется с помощью валидационных данных. Вместо этого указанные данные используются после каждой n-ой эпохи для
оценивания возможного результата на тестовых данных. Как и тестовые данные, валидационные данные пропускаются через модель
без обновления модельных параметров (в режиме предсказательного вывода), после чего измеряются потеря и точность.
```

```
Если набор данных очень мал и  использование еще меньшего объема данных для тренировки оказывает негативное влияние, то можно применить перекрестную валидацию. Вместо того чтобы часть тренировочных данных, на которых модель никогда не будет тренироваться, с самого начала откладывать в сторону, в каждой эпохе выполняется случайная разбивка. В начале каждой эпохи примеры для валидации отбираются в случайном порядке и не используются для тренировки в этой эпохе, а вместо этого используются для валидационного теста. Но поскольку отбор является случайным, некоторые или все примеры будут появляться среди тренировочных данных в других эпохах
```

<img src="images/cross_valid.png" alt="сross_validation" height=50% width=50%>

#### Потеря

```
    Ранее все наше внимание было сосредоточено на точности. Но во время тренировки вы увидите еще одну метрику – среднюю по batch-ам потерю (ошибку) как для тренировочных данных, так и для тестовых. В идеале мы хотели бы видеть неуклонное увеличение точности в расчете на
эпоху. Но мы также можем увидеть отрезки эпох, для которых точность выходит на плато или даже колеблется.
    Важно то, что мы видим стабильное снижение потери. Плато или колебания в этом случае происходят из-за того, что мы находимся
рядом с линиями линейной разделимости, или не полностью перешли линию, но приближаемся, о чем свидетельствует уменьшение потери.
    
```

```
Давайте рассмотрим пример. Допустим, вы создаете классификатор собак и кошек. В выходном слое у вас есть два узла: один для кошек и один для собак. Допустим, что в конкретном batch-е, когда модель неправильно классифицирует собаку как кошку, выходные значения (уровень уверенности) равны 0.6 для кошки и 0.4 для собаки. В последующем batch-е, когда модель снова неправильно классифицирует собаку как кошку, выходные значения равны 0.55 (кошка) и 0.45 (собака). Теперь значения ближе к фундаментальным истинам, и, следовательно, потеря уменьшается, но они все еще не преодолели порог 0.5, поэтому точность пока не изменилась. Затем допустим, что в еще одном последующем пакете выходные значения для изображения собаки равны 0.49 (кошка) и 0.51 (собака); потеря еще больше уменьшилась, и поскольку мы
пересекли порог 0.5, точность повысилась.
```

## 5 Схождение

<img src="images/convergence.png" alt="convergence" height=50% width=50%>

```
    Ранние предположения о  тренировке заключались в  том, что чем больше раз вводить тренировочные данные в модель, тем выше будет
точность. Мы обнаружили, в особенности в более крупных и сложных сетях, что в определенный момент точность будет деградировать. Сегодня мы стремимся достигать схождения на приемлемом локальном оптимуме, основываясь на том, как модель будет использоваться в приложении. Если излишне натренировать нейронную сеть, то может произойти следующее:
    1) нейронная сеть становится переподогнанной к тренировочным данным, демонстрируя повышенную точность на тренировочных данных, но показывая деградирующую точность на тестовых данных;
    2) в более глубоких нейронных сетях слои будут учиться неравномерно и иметь разные скорости обучения. Отсюда, поскольку
некоторые слои продолжают обучаться (работают в направлении схождения), другие уже могут начать расходиться;
    3) продолжение тренировки может привести к тому, что нейронная сеть выскочит из одного локального оптимума и начнет сходиться на другом, менее точном.

```

## 6 Контрольные точки и ранняя остановка